In [31]:
from selenium import webdriver
from PIL import Image
import PIL
import io
import base64
import time
import re
from slugify import slugify
from webdriverdownloader import GeckoDriverDownloader
from pathlib import Path
from IPython.display import display, HTML

gdd = GeckoDriverDownloader()
geckodriver = gdd.download_and_install("v0.26.0")[1]

# See https://github.com/ouseful-template-repos/binder-selenium-demoscraper
# Note that currently isn't working on Binder -- perhaps related to this https://github.com/mozilla/geckodriver/issues/1559

Works with NLNZ, IA, & ...

In [32]:
URL = 'https://ndhadeliver.natlib.govt.nz/webarchive/wayback/20140922035225if_/http://www.shetlandwellington.org.nz/'
    
def get_full_page_screenshot(url, save_width=200):
    '''
    Gets a full page screenshot of the supplied url.
    By default resizes the screenshot to a maximum width of 200px.
    Provide a 'save_width' value to change this.
    '''
    # print(url)
    date_str, site = re.search(r'/(\d{14})(?:if_|mp_)*/https*://(.+/)', url).groups()
    ss_file = Path('screenshots', f'{slugify(site)}-{date_str}-{save_width}.png')
    options = webdriver.FirefoxOptions()
    options.headless = True
    driver = webdriver.Firefox(executable_path=geckodriver, options=options)
    driver.get(url)
    # Give some time for everything to load
    time.sleep(5)
    # Can just use maximize_window() instead of the below (only in Geckodriver?)
    # S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
    # driver.set_window_size(capture_width, S('Height') + 50) # May need manual adjustment
    driver.maximize_window()
    current_width = driver.get_window_size()['width']
    try:
        ss = driver.find_element_by_tag_name('body').screenshot_as_base64
    except NoSuchElementException:
        ss = driver.find_element_by_tag_name('frameset').screenshot_as_base64
    driver.quit()
    img = Image.open(io.BytesIO(base64.b64decode(ss)))
    ratio = save_width / current_width
    (width, height) = (round(img.width * ratio), round(img.height * ratio))
    resized_img = img.resize((width, height), PIL.Image.LANCZOS)
    resized_img.save(ss_file)
    display(HTML(f'<img src="{ss_file}">'))
    

In [33]:
get_full_page_screenshot(URL)